In [1]:
from tabulate import tabulate
import numpy as np
import time
import re
import pandas as pd
from requests_html import HTMLSession
from time import localtime, strftime
import locale
locale.setlocale(locale.LC_ALL, '')
pd.set_option("display.unicode.ambiguous_as_wide",True)
pd.set_option("display.unicode.east_asian_width",True)
session = HTMLSession()

In [2]:
def str_to_int(str1):
    str1 = str1.replace(",", "")
    return int(float(str1))
def periodic_work_2():
    col_names=["股票","当前价格","价格变化","当前交易量","三月平均交易量"]
    stock_names=["NASDAQ","S&P 500","FACA","MSFT","AMD","DOCN","PATH","DDOG","EPAM","NVDA","SQ","AFRM","S","NET","UPST","TWLO","CRWD","HCP","MDB","GOOG","AAPL"]
    retval=np.zeros((len(stock_names),len(col_names)),dtype=object)
    
    session = HTMLSession()
    url = "https://finance.yahoo.com/quote/%5EIXIC?p=^IXIC&.tsrc=fin-srch"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Currency in USD", all_page)
    start=x.span()[0]
    temp=all_page[start:start+150]
    temp.split("\n")
    all_info=["NASDAQ",temp.split("\n")[2],temp.split("\n")[4][1:-1]]
    vol=re.search(r"Volume",all_page).span()[0]
    temp=all_page[vol:vol+200]
    all_info+=[temp.split("\n")[1],temp.split("\n")[7]]
    retval[0,:]=all_info
    
    session = HTMLSession()
    url = "https://finance.yahoo.com/quote/%5EGSPC?p=^GSPC&.tsrc=fin-srch"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Currency in USD", all_page)
    start=x.span()[0]
    temp=all_page[start:start+150]
    temp.split("\n")
    all_info=["S&P 500",temp.split("\n")[2],temp.split("\n")[4][1:-1]]
    vol=re.search(r"Volume",all_page).span()[0]
    temp=all_page[vol:vol+200]
    all_info+=[temp.split("\n")[1],temp.split("\n")[7]]
    retval[1,:]=all_info
    
    for i in range(2,len(stock_names)):
        retval[i,:]=get_stock_info(stock_names[i])
    
    #print(strftime("日期：%Y-%m-%d 时间：%H:%M:%S", localtime()))
    #print()
    df_r=pd.DataFrame(retval,columns=col_names).reset_index(drop=True)
    #prhttp://localhost:8888/notebooks/workspace/asg/pa_chong/stocks_monitor.ipynb#int(tabulate(df_r, headers = 'keys', tablefmt = 'psql'))
    #print("{: >5} {: >25} {: >15} {: >15} {: >15} {: >15}".format(*col_names))
    #for row in retval:
    #    print("{: >5} {: >25} {: >15} {: >15} {: >15} {: >15}".format(*row))
    #interval should be an integer, the number of seconds to wait
    df_r.index = np.arange(1, len(df_r)+1)
    df_r["当前交易量/三月平均交易量"]=[str_to_int(df_r["当前交易量"][i])/str_to_int(df_r["三月平均交易量"][i]) for i in range(1,len(stock_names)+1)]
    return df_r

In [3]:
import math 
#XPEV $24.97, BZ $25.46
def get_stock_history(token):
    url = "https://finance.yahoo.com/quote/"+token+"/history?p="+token
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Volume", all_page)
    start=x.span()[0]
    temp=all_page[start:start+550]
    first_day=temp.split("\n")[14]
    sec_day=temp.split("\n")[21]
    third_day=temp.split("\n")[28]
    if first_day == "-":
        first_day="0"
    if sec_day == "-":
        sec_day="0"
    if third_day == "-":
        third_day="0"
    all_info=[str_to_int(first_day),str_to_int(sec_day),str_to_int(third_day)]
    three_day_avg=sum(all_info)/len(all_info)
    return three_day_avg
def get_stock_info(token):
    url = "https://finance.yahoo.com/quote/"+token+"?p="+token+"&.tsrc=fin-srch"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"trend2W", all_page)
    start=x.span()[0]
    temp=all_page[start:start+150] # truncate the string to texts we want
    start = re.search("(\d+)\.(\d+)",temp).span()[0] # find the first number (price)
    temp = temp[start:] # truncate the string from the first number
    temp = temp.split("\n") # split the string by newline
    all_info=[token,temp[0],temp[2][1:-1]] # get the three elements
    vol=re.search(r"Volume",all_page).span()[0]
    temp=all_page[vol:vol+200]
    all_info+=[temp.split("\n")[1],temp.split("\n")[3]]
    return all_info
def highlight_max_2(x):
    return ['font-weight: bold' if v == x.loc[1] or v == x.loc[2] else '' for v in x]
def NASDAQ():
    url="https://finance.yahoo.com/quote/%5EIXIC/history?p=%5EIXIC"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Volume", all_page)
    start=x.span()[0]
    temp=all_page[start:start+250]
    all_info=[str_to_int(temp.split("\n")[14]),str_to_int(temp.split("\n")[21]),str_to_int(temp.split("\n")[21])]
    three_day_avg=sum(all_info)/len(all_info)
    return three_day_avg
def NASDAQ_three_day():
    url="https://finance.yahoo.com/quote/%5EIXIC/history?p=%5EIXIC"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Volume", all_page)
    start=x.span()[0]
    temp=all_page[start:start+500]
    all_info=[str_to_int(temp.split("\n")[14]),str_to_int(temp.split("\n")[21]),str_to_int(temp.split("\n")[28])]
    three_day_avg=sum(all_info)/len(all_info)
    return three_day_avg
def sp500_three_day():
    url="https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"
    r = session.get(url)
    all_page=r.html.text
    x = re.search(r"Volume", all_page)
    start=x.span()[0]
    temp=all_page[start:start+500]
    all_info=[str_to_int(temp.split("\n")[14]),str_to_int(temp.split("\n")[21]),str_to_int(temp.split("\n")[28])]
    three_day_avg=sum(all_info)/len(all_info)
    return three_day_avg
def others_three_day():
    return [round(get_stock_history(df["股票"][i])) for i in range(3,len(df["股票"])+1)]

In [4]:
def select_col(x):
    
    c1 = 'color: green'
    c2 = '' 
    c3 = "color: red"
    #compare columns
    mask=[]
    mask2=[]
    for i in range(1,len(x["当前交易量"])+1):
        mask.append(str_to_int(x["当前交易量"][i])>str_to_int(x["三月平均交易量"][i]))
        mask2.append(str_to_int(x["当前交易量"][i])<=str_to_int(x["三月平均交易量"][i]))
    #DataFrame with same index and columns names as original filled empty strings
    df1 =  pd.DataFrame(c2, index=x.index, columns=x.columns)
    #modify values of df1 column by boolean mask
    df1.loc[mask, '当前交易量'] = c1
    df1.loc[mask2, '当前交易量'] = c3
    return df1
def style_specific_cell(x,i,j):

    color = 'background-color: lightgreen'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[i, j] = color
    return df1

t_styles = [{'selector': 'tr:hover',
      'props': [('background-color', 'yellow')]}]
while True:
    df=periodic_work_2()
    
    df["temp_str"]=["{:.2%}".format(df["当前交易量/三月平均交易量"][i]) for i in range(1,len(df["当前交易量/三月平均交易量"])+1)]
    df["当前交易量/三月平均交易量"]=df["temp_str"]
    df=df.drop(columns=["temp_str"])
    df.index = np.arange(1, df.shape[0]+1)
    #df["3天平均交易量"]=[int(get_stock_history(df["股票"][i])) for i in range(1,len(df["股票"])+1)]
    df["temp"]=df["价格变化"].astype(str).str[:-1][1:].astype(str)
    excluded=df[(df["股票"] == "NASDAQ") | (df["股票"] == "S&P 500")]
    included=df[(df["股票"] != "NASDAQ") & (df["股票"] != "S&P 500")]
    sort_price = included.sort_values(by="temp",ascending=False)
    df=pd.concat([excluded,sort_price]).drop(columns=["temp"])
    all_three_days=[round(NASDAQ_three_day()),round(sp500_three_day())]
    all_three_days=all_three_days+others_three_day()
    df["三天平均交易量"]=['{0:,}'.format(elem) for elem in all_three_days]
    df.index = np.arange(1, len(df)+1)
    df=df[["股票","当前价格","价格变化","当前交易量","三天平均交易量","三月平均交易量","当前交易量/三月平均交易量"]]
    df=df.drop(columns=["当前交易量","三天平均交易量","三月平均交易量"])
    #df=df.rename(columns={"当前交易量": "今天交易量", "当前交易量/平均交易量": "今日交易量/三月平均交易量"})
    print(strftime("Date: %Y-%m-%d %H:%M:%S", localtime()))
    display(df.style.\
    applymap(lambda x: "color: red" if float(x[:-1])<0 else "color: green",subset=["价格变化"]).\
    apply(highlight_max_2).\
    # apply(select_col, axis=None).\
    set_caption("重点关注观测").set_table_styles(t_styles).\
    apply(lambda x:style_specific_cell(x,16,2), axis=None)
    )
    break
    time.sleep(1800)

Date: 2022-07-22 10:01:47


,股票,当前价格,价格变化,当前交易量/三月平均交易量
1,NASDAQ,"11,970.67",-0.74%,14.73%
2,S&P 500,"3,987.00",-0.30%,8.70%
3,FACA,9.83,0.00%,0.08%
4,UPST,26.84,-5.66%,10.09%
5,DOCN,41.02,-4.18%,7.29%
6,AFRM,28.71,-3.50%,14.82%
7,NVDA,174.71,-3.21%,15.88%
8,HCP,34.94,-2.92%,5.61%
9,PATH,20.41,-2.81%,4.94%
10,AMD,88.63,-2.70%,11.97%


In [5]:
df

,股票,当前价格,价格变化,当前交易量/三月平均交易量
1,NASDAQ,"11,970.67",-0.74%,14.73%
2,S&P 500,"3,987.00",-0.30%,8.70%
3,FACA,9.83,0.00%,0.08%
4,UPST,26.84,-5.66%,10.09%
5,DOCN,41.02,-4.18%,7.29%
6,AFRM,28.71,-3.50%,14.82%
7,NVDA,174.71,-3.21%,15.88%
8,HCP,34.94,-2.92%,5.61%
9,PATH,20.41,-2.81%,4.94%
10,AMD,88.63,-2.70%,11.97%


In [1]:
df.style.applymap(lambda x: "color: red" if float(x[:-1])<0 else "color: green",subset=["价格变化"]).apply(highlight_max_2).set_caption("重点关注观测").set_table_styles(t_styles).apply(lambda x:style_specific_cell(x,16,2), axis=None)

NameError: name 'df' is not defined

In [6]:
t_styles = [{'selector': 'tr:hover',
      'props': [('background-color', 'yellow')]}]
display(df.style.\
applymap(lambda x: "color: red" if float(x[:-1])<0 else "color: green",subset=["价格变化"]).\
apply(highlight_max_2).\
# apply(select_col, axis=None).\
set_caption("重点关注观测").set_table_styles(t_styles).\
apply(lambda x:style_specific_cell(x,16,2), axis=None)
)

,股票,当前价格,价格变化,当前交易量/三月平均交易量
1,NASDAQ,"11,970.67",-0.74%,14.73%
2,S&P 500,"3,987.00",-0.30%,8.70%
3,FACA,9.83,0.00%,0.08%
4,UPST,26.84,-5.66%,10.09%
5,DOCN,41.02,-4.18%,7.29%
6,AFRM,28.71,-3.50%,14.82%
7,NVDA,174.71,-3.21%,15.88%
8,HCP,34.94,-2.92%,5.61%
9,PATH,20.41,-2.81%,4.94%
10,AMD,88.63,-2.70%,11.97%
